<h1>Automated Flood Detection based on Satellite Images</h1>
<br>
This project focuses on flood detection using only the SEN2 data from the SEN12-FLOOD dataset. The goal is to process Sentinel-2 satellite images, specifically four spectral bands: Band2 (blue), Band3 (green), Band4 (red), and Band8 (infrared). These bands are important for distinguishing between land, water, and other surface features. The process involves organizing the data, checking for empty images, and stacking the bands together for further analysis, which will be used for flood detection tasks.
<br>
<hr>
Dataset:<br>
Clément Rambour, Nicolas Audebert, Elise Koeniguer, Bertrand Le Saux, Michel Crucianu, Mihai Datcu. (2020). SEN12-FLOOD : a SAR and Multispectral Dataset for Flood Detection . IEEE Dataport. https://dx.doi.org/10.21227/w6xz-s898

Download (after free registration) - 12,2Gb: https://ieee-dataport.org/open-access/sen12-flood-sar-and-multispectral-dataset-flood-detection


In [1]:
import rasterio
import pandas as pd
import os
import shutil
import cv2
import numpy as np

<h1>Functions</h1>
<ul>
<li>check_empty_img(url)</li>
<li>remove_folders(path)</li>
<li>create_dir_structure(flist)</li>
<li>stack_bands(path)</li>
</ul>

In [2]:
def check_empty_img(url):
    
    path = os.path.join(url, 'B02.tif')

# Load the image in grayscale mode (0)
    image = cv2.imread(path,0)
  
 # Return True if all pixels are zero (empty image), otherwise False
    if (cv2.countNonZero(image) == 0):
        return  True
    else:
        return  False
  

In [3]:
def remove_folders(path):
    # Delete the folder and all its contents
    shutil.rmtree(path)
    return

In [4]:
def create_dir_structure(flist):
    for folder in flist:
        prefix = folder[:11]    # Extract prefix from folder name
        id = folder[11:]        # Extract id from folder name

        if len(id) > 4: continue # Skip if id length is more than 4, because the id can be at most 4 digits

        for file in os.listdir(folder):

            if file.startswith('S1'): 
                os.remove(os.path.join(prefix + id, file)) # Remove files starting with 'S1'
                continue
            # If the file is a spectral band (B02, B03, B04, B08)
            if file.endswith('B02.tif') or file.endswith('B03.tif') or file.endswith('B04.tif') or file.endswith('B08.tif'):
                date = file[3:13] # Extract date from file name
                newFolder =  os.path.join(prefix,'S2_' + id + '_'+ date) # Create new folder name

                if os.path.isdir(newFolder):
                    shutil.move(os.path.join(folder, file), os.path.join(newFolder, file[14:])) # Move file if folder exists
                else:
                    os.mkdir(newFolder)     # Create folder if it doesn't exist
                    shutil.move(os.path.join(folder, file), os.path.join(newFolder, file[14:]))  # Move file

        remove_folders(folder) # Remove the original folder after moving files


In [5]:
def stack_bands(path):
    print(path)

    
    band_list = ['B02.tif', 'B03.tif', 'B04.tif', 'B08.tif']
    try:
        
        # Read metadata from the first band (B02.tif) 
        with rasterio.open(os.path.join(path, band_list[0])) as src0:
            meta = src0.meta # Extract metadata from the first band
        
        # Update metadata to reflect the number of bands (layers) 
        meta.update(count = len(band_list))

       # Create a new stack file and write each band to it
        with rasterio.open(os.path.join(path, 'stack.tif'), 'w', **meta) as dst:
            for id, layer in enumerate(band_list, start=1):
                with rasterio.open(os.path.join(path, layer)) as src1:
                    dst.write_band(id, src1.read(1)) # Write each band to the stack file
    except:
        print("Folder with no Data") # Handle cases where the folder has no valid data 
        remove_folders(path) # Remove the folder if there's an issue
        
        pass # Continue without stopping on error 

    

<h2> Data operations </h2>

In [6]:
# Create a list of all folders containing spectral bands


flist = []

rootdir = os.path.normpath('./SEN12FLOOD')  # Get the absolute path for the root directory
#rootdir = './SEN12FLOOD' 
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file) # Create the full path for each file/folder
    if os.path.isdir(d):    # Check if it's a folder
        flist.append(d)     # Add folder to the list
        
# Print the total number of folders found        
print(f"The number of folders are currently = {len(flist)}")

The number of folders are currently = 1782


In [7]:
# Call the function to create the directory structure for the initial folder list (can run up to 1-2 min)
create_dir_structure(flist)


In [8]:
# Create a new list of folders that start with 'S2_'

flist = []
#rootdir = './SEN12FLOOD'
rootdir = os.path.normpath('./SEN12FLOOD')
for file in os.listdir(rootdir):
     if 'S2_' in file:
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            flist.append(d)
        
        
print(f"The number of folders are currently = {len(flist)}")


The number of folders are currently = 1782


In [9]:
# Iterate through all folders and create a new image with 4 spectral bands:
# Band2 (blue), Band3 (green), Band4 (red), and Band8 (infrared)
#This process can take up to 2-5 minutes to complete


for folder_path in flist:
    empty = check_empty_img(folder_path) # Check if the folder contains empty images (all pixels are zero)
    if empty:
        
        print("The images inside the current folder are empty - zero")
        remove_folders(folder_path) # Remove the folder if images are empty 
    else:
        stack_bands(folder_path) # Stack the spectral bands if images are valid

    
    

SEN12FLOOD\S2_0084_2018-12-26
SEN12FLOOD\S2_0084_2018-12-31
SEN12FLOOD\S2_0084_2019-01-05
SEN12FLOOD\S2_0084_2019-01-10
SEN12FLOOD\S2_0084_2019-01-30
SEN12FLOOD\S2_0084_2019-02-04
SEN12FLOOD\S2_0084_2019-02-14
SEN12FLOOD\S2_0085_2018-12-26
SEN12FLOOD\S2_0085_2018-12-31
SEN12FLOOD\S2_0085_2019-01-05
SEN12FLOOD\S2_0085_2019-01-10
SEN12FLOOD\S2_0085_2019-01-30
SEN12FLOOD\S2_0085_2019-02-04
SEN12FLOOD\S2_0085_2019-02-14
SEN12FLOOD\S2_0086_2018-12-26
SEN12FLOOD\S2_0086_2018-12-31
SEN12FLOOD\S2_0086_2019-01-05
SEN12FLOOD\S2_0086_2019-01-10
SEN12FLOOD\S2_0086_2019-01-30
SEN12FLOOD\S2_0086_2019-02-04
SEN12FLOOD\S2_0086_2019-02-14
SEN12FLOOD\S2_0088_2018-12-26
SEN12FLOOD\S2_0088_2018-12-31
SEN12FLOOD\S2_0088_2019-01-05
SEN12FLOOD\S2_0088_2019-01-10
SEN12FLOOD\S2_0088_2019-01-30
SEN12FLOOD\S2_0088_2019-02-04
SEN12FLOOD\S2_0088_2019-02-14
SEN12FLOOD\S2_0089_2018-12-26
SEN12FLOOD\S2_0089_2018-12-31
SEN12FLOOD\S2_0089_2019-01-05
SEN12FLOOD\S2_0089_2019-01-10
SEN12FLOOD\S2_0089_2019-01-30
SEN12FLOOD

JSON Metadata Processing

In [10]:
import json
import os

# Step 2: Open and load the JSON file
with open('./SEN12FLOOD/S2list.json', 'r') as file:
    data = json.load(file)

print( data)



{'0063': {'1': {'date': '2019-02-04', 'FLOODING': False, 'FULL-DATA-COVERAGE': True, 'filename': 'S2_2019-02-04'}, 'count': 1, 'folder': '0063', 'geo': {'type': 'Polygon', 'coordinates': [[[28.29722, -15.382762], [28.297507, -15.429039], [28.345216, -15.428755], [28.344918, -15.382479], [28.29722, -15.382762]]]}}, '0200': {'1': {'date': '2019-02-23', 'FLOODING': False, 'FULL-DATA-COVERAGE': True, 'filename': 'S2_2019-02-23'}, '2': {'date': '2019-02-28', 'FLOODING': False, 'FULL-DATA-COVERAGE': True, 'filename': 'S2_2019-02-28'}, '3': {'date': '2019-03-10', 'FLOODING': False, 'FULL-DATA-COVERAGE': True, 'filename': 'S2_2019-03-10'}, '4': {'date': '2019-03-25', 'FLOODING': True, 'FULL-DATA-COVERAGE': True, 'filename': 'S2_2019-03-25'}, '5': {'date': '2019-04-04', 'FLOODING': True, 'FULL-DATA-COVERAGE': True, 'filename': 'S2_2019-04-04'}, '6': {'date': '2019-04-14', 'FLOODING': True, 'FULL-DATA-COVERAGE': True, 'filename': 'S2_2019-04-14'}, '7': {'date': '2019-04-24', 'FLOODING': True, 'F

In [11]:
for location_id, location_data in data.items():
    for location_date, date_data in location_data.items():
        if location_date == 'geo' or location_date =='count' or location_date =='folder': continue
        folder = os.path.join(rootdir, 'S2_' + location_id + '_' + date_data['date'])
        #print (folder, date_data['FLOODING'])
        file = os.path.join(folder, 'flooding.txt')
        print(file)


        if os.path.isdir(folder):
            with open(file, 'w') as f:
                f.write(f"{date_data['FLOODING']}\n")


SEN12FLOOD\S2_0063_2019-02-04\flooding.txt
SEN12FLOOD\S2_0200_2019-02-23\flooding.txt
SEN12FLOOD\S2_0200_2019-02-28\flooding.txt
SEN12FLOOD\S2_0200_2019-03-10\flooding.txt
SEN12FLOOD\S2_0200_2019-03-25\flooding.txt
SEN12FLOOD\S2_0200_2019-04-04\flooding.txt
SEN12FLOOD\S2_0200_2019-04-14\flooding.txt
SEN12FLOOD\S2_0200_2019-04-24\flooding.txt
SEN12FLOOD\S2_0200_2019-04-29\flooding.txt
SEN12FLOOD\S2_0004_2018-12-15\flooding.txt
SEN12FLOOD\S2_0004_2019-01-19\flooding.txt
SEN12FLOOD\S2_0004_2019-01-21\flooding.txt
SEN12FLOOD\S2_0004_2019-01-24\flooding.txt
SEN12FLOOD\S2_0004_2019-01-26\flooding.txt
SEN12FLOOD\S2_0004_2019-01-31\flooding.txt
SEN12FLOOD\S2_0004_2019-02-05\flooding.txt
SEN12FLOOD\S2_0004_2019-02-13\flooding.txt
SEN12FLOOD\S2_0004_2018-12-17\flooding.txt
SEN12FLOOD\S2_0004_2018-12-20\flooding.txt
SEN12FLOOD\S2_0004_2018-12-22\flooding.txt
SEN12FLOOD\S2_0004_2018-12-30\flooding.txt
SEN12FLOOD\S2_0004_2019-01-04\flooding.txt
SEN12FLOOD\S2_0004_2019-01-06\flooding.txt
SEN12FLOOD\

In [12]:
def getLabel(path):
    filepath = os.path.normpath(os.path.join(path, 'flooding.txt'))
    with open(filepath, 'r') as file:
        if(file.readline() == 'False'):
            return 0
        else:
            return 1

In [13]:
print(getLabel("SEN12FLOOD\S2_0_2019-01-31"))

FileNotFoundError: [Errno 2] No such file or directory: 'SEN12FLOOD\\S2_0_2019-01-31\\flooding.txt'

Augmentation

pip install numpy torch torchvision tifffile

In [19]:
import os
import numpy as np
import torch
from torchvision import transforms
import tifffile as tiff  # TIFF fájlok betöltéséhez és kezeléséhez


In [20]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),  # 15 fokon belüli véletlenszerű forgatás
])

In [21]:
def augment_images_in_directory(input_dir):
    # Fájlok bejárása a megadott mappában
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file == 'stack.tif':  # Csak a 'stack.tif' képekre fókuszálunk
                file_path = os.path.join(root, file)
                
                # TIFF kép betöltése 4 csatornával (HxWxC)
                img = tiff.imread(file_path)
                
                # Kép tensorrá alakítása és átrendezése (CxHxW formátum PyTorch számára)
                img_tensor = torch.from_numpy(img).permute(2, 0, 1).float()
                
                # Augmentáció alkalmazása
                img_transformed = train_transform(img_tensor)
                
                # Kép visszaalakítása (HxWxC formátumba numpy-hoz)
                img_transformed = img_transformed.permute(1, 2, 0).numpy().astype(np.uint16)
                
                # Augmentált kép mentése új névvel: 'astack.tif'
                save_path = os.path.join(root, 'astack.tif')
                tiff.imwrite(save_path, img_transformed)
                
                print(f"Kép augmentálva és mentve: {save_path}")

# Fő futtatás
input_directory = './SEN12FLOOD'  # A gyökérmappa, ahol a mappák találhatóak
augment_images_in_directory(input_directory)

Kép augmentálva és mentve: ./SEN12FLOOD\S2_0084_2018-12-26\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0084_2018-12-31\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0084_2019-01-05\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0084_2019-01-10\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0084_2019-01-30\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0084_2019-02-04\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0084_2019-02-14\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0085_2018-12-26\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0085_2018-12-31\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0085_2019-01-05\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0085_2019-01-10\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0085_2019-01-30\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0085_2019-02-04\astack.tif
Kép augmentálva és mentve: ./SEN12FLOOD\S2_0085_2019-02-14\astack.tif
Kép augmentálva és m

Data loading

In [3]:
flist = []
rootdir = os.path.normpath('./SEN12FLOOD')
for file in os.listdir(rootdir):
     if 'S2_' in file:
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            flist.append(d)
        
        
print(f"The number of folders are currently = {len(flist)}")

The number of folders are currently = 2039


In [20]:
def load_data():

    images = []
    labels = []

    for folder in flist:
        imagePath = os.path.join(folder, 'stack.tif')
        image = cv2.imread(imagePath)
        images.append(image)

        label = getLabel(folder)
        labels.append(label)

    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'int32')

    return images, labels
    

In [21]:
train_images, train_labels = load_data()

In [22]:
train_images.shape

(2039, 512, 512, 3)